このノートブックは、次のページの内容に基づいて作られています。

http://gluon.mxnet.io/chapter01_crashcourse/probability.html#Basic-probability-theory

# 確率論の基礎

必要なライブラリを読み込みます。

In [1]:
require 'mxnet'

true

サイコロの各目が出る確率の理論値を変数 `probabilities` に入れます。

In [2]:
probabilities = MXNet::NDArray.ones(6) / 6


[0.166667, 0.166667, 0.166667, 0.166667, 0.166667, 0.166667]
<MXNet::NDArray 6 @cpu(0)>

この確率にしたがって、サイコロを振ってみましょう。
MXNet では、`MXNet::NDArray.sample_multinomial` を使うと多項分布からサンプリングできます。

単一のサンプルを取り出してみましょう。確率のベクトルを渡すだけです。

In [3]:
MXNet::NDArray.sample_multinomial(probabilities)


[3]
<MXNet::NDArray 1 @cpu(0)>

`MXNet::NDArray.sample_multinomial(probabilities)` を呼び出すたびに、確率変数を得ることができます。そのため、 `Array.new(10) { ... }` のような記法を使って複数回これを呼び出した結果を集めれば、同じ分布から複数のサンプルを生成したことになります。しかし、このやり方は遅いです。

`sample_multinomial` は複数のサンプルを一度に生成することに対応しています。以下のように、`shape:` キーワードで結果の数値配列の形を指定するだけです。

In [4]:
MXNet::NDArray.sample_multinomial(probabilities, shape: [10])


[3, 4, 5, 3, 5, 3, 5, 2, 3, 3]
<MXNet::NDArray 10 @cpu(0)>

In [5]:
MXNet::NDArray.sample_multinomial(probabilities, shape: [5, 10])


[[2, 2, 1, 5, 0, 5, 1, 2, 2, 4], 
 [4, 3, 2, 3, 2, 5, 5, 0, 2, 0], 
 [3, 0, 2, 4, 5, 4, 0, 5, 5, 5], 
 [2, 4, 4, 2, 3, 4, 4, 0, 4, 3], 
 [3, 0, 3, 5, 4, 3, 0, 2, 2, 1]]
<MXNet::NDArray 5x10 @cpu(0)>

では、1000回サイコロを振った結果を生成してみましょう。

In [6]:
rolls = MXNet::NDArray.sample_multinomial(probabilities, shape: [1000])
nil # <- This nil avoids to display the last result

In [7]:
counts = MXNet::NDArray.zeros([6, 1000])
totals = MXNet::NDArray.zeros([6])
rolls.each_with_index do |roll, i|
  totals[roll.as_scalar.to_i] += 1
  counts[0..-1, i] = totals
end
nil

In [8]:
totals / 1000


[0.167, 0.168, 0.175, 0.159, 0.158, 0.173]
<MXNet::NDArray 6 @cpu(0)>

In [9]:
counts


[[0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...], 
 [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...], 
 [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 5, 5, ...], 
 [0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, ...], 
 [0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, ...], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 3, 3, ...]]
<MXNet::NDArray 6x1000 @cpu(0)>

In [10]:
counts[0..-1, -3..-1]


[[165, 166, 167], 
 [168, 168, 168], 
 [175, 175, 175], 
 [159, 159, 159], 
 [158, 158, 158], 
 [173, 173, 173]]
<MXNet::NDArray 6x3 @cpu(0)>

In [11]:
x = MXNet::NDArray.arange(1000).reshape([1, 1000]) + 1
estimates = counts / x
IRuby.display estimates[0..-1, 0]
IRuby.display estimates[0..-1, 1]
IRuby.display estimates[0..-1, 100]
nil


[0, 1, 0, 0, 0, 0]
<MXNet::NDArray 6 @cpu(0)>


[0, 0.5, 0, 0, 0.5, 0]
<MXNet::NDArray 6 @cpu(0)>


[0.19802, 0.158416, 0.178218, 0.188119, 0.128713, 0.148515]
<MXNet::NDArray 6 @cpu(0)>

In [12]:
require 'rbplotly'

indices = [*0...estimates.shape[1]]
plot = Plotly::Plot.new(data: [
  { x: indices, y: estimates[0, 0..-1].to_a, name: 'Estimated P(die=1)' },
  { x: indices, y: estimates[1, 0..-1].to_a, name: 'Estimated P(die=2)' },
  { x: indices, y: estimates[2, 0..-1].to_a, name: 'Estimated P(die=3)' },
  { x: indices, y: estimates[3, 0..-1].to_a, name: 'Estimated P(die=4)' },
  { x: indices, y: estimates[4, 0..-1].to_a, name: 'Estimated P(die=5)' },
  { x: indices, y: estimates[5, 0..-1].to_a, name: 'Estimated P(die=6)' },
])
plot.show

#<Plotly::Offline::HTML:0x00007f96efa2bbf0 @id="73e7670c-fddf-49e4-a7be-b9880a10e095", @data=[{:x=>[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 

#<CZTop::Socket::PUB:0x7f96f43427c0 last_endpoint="tcp://127.0.0.1:63811">

In [13]:
unless File.exist?('train-images-idx3-ubyte') &&
       File.exist?('train-labels-idx1-ubyte')
  system("wget http://data.mxnet.io/mxnet/data/mnist.zip")
  system("unzip -x mnist.zip")
end

In [14]:
ycount = MXNet::NDArray.ones([10])
xcount = MXNet::NDArray.ones([784, 10])  # 784 == 28*28

mnist_iter = MXNet::IO::MNISTIter.new(batch_size: 1)
mnist_iter.each do |batch|
  x = batch.data[0].reshape([784])
  y = batch.label[0].to_i
  ycount[y] += 1
  xcount[0..-1, y] += x
end

0.upto(9) do |i|
  xcount[0..-1, i] = xcount[0..-1, i] / ycount[i]
end

py = ycount / MXNet::NDArray.sum(ycount)


[0.0987169, 0.112365, 0.0993001, 0.102183, 0.0973671, 0.0903516, 0.0986336, ...]
<MXNet::NDArray 10 @cpu(0)>

In [17]:
require 'chunky_png'
require 'base64'

def imshow(ary)
  height, width = ary.shape
  fig = ChunkyPNG::Image.new(width, height, ChunkyPNG::Color::TRANSPARENT)
  ary = ((ary - ary.min) / ary.max) * 255
  0.upto(height - 1) do |i|
    0.upto(width - 1) do |j|
      v = ary[i, j].round
      fig[j, i] = ChunkyPNG::Color.rgba(v, v, v, 255)
    end
  end

  src = 'data:image/png;base64,' + Base64.strict_encode64(fig.to_blob)
  IRuby.display "<img src='#{src}' width='#{width*2}' height='#{height*2}' />", mime: 'text/html'
end

0.upto(9) do |i|
  imshow(xcount[0..-1, i].reshape([28, 28]).to_narray)
end

py

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAACFElEQVR4nK3Sz2/SYBgH8L5v25cC8mstBtj4oYgIZERnzLaMhcRdpmHGmydvxoN3PZv4J3jzL1hi8GZ20JiZqVk0gQiW6MZAamBIhwPHoJa2r8FQGJedfK6ffL95nuSBxCkD/wcCSFI0QjRFQjCJANKM3RtfvnnrxlLMbUHgBAJAmZyhhdTd+w/u3VmJe8wUGOHAuOBcIpmYv5ZYSc6HzxrGSEDG4YtdmfXZgEo7L8UjXis5TtIWdyh6wUn83Mnzdcob8k9ROgJo5PzhIKfWch/ebGYaaNrHolGStnqC5zhiP//x3fvtTEmyujnDKMlw/vNuusFnsoViqVzt0FbHYKN/CM2uwMyZTjHHlxutVqvdxcg0SkL7jI/VqoWvwmFP6quKomE42pbmPC5ju7RbPZIVTDEMpUgyodci1mXDYqV2JGsAWViWkTsdGQ/R4LAbJLH+W8YQ2Tx+N+qITUmvRSaGlI97KqSNbCBykdPEWqOnI4kxQSKzxcb5olcvB5hmRTiQhrVA6/ZUszccnZ1bXF6MWH6V9n60+sMklsTGsTV2fW0ttZqMmg92CsV6V9Nre7VdQZpeSt1eXfATwucMX2n38Rjz21lBdbimVOHT261s+XBwyBCVZmHzxXp64/XL9Hr6VeZ7S9bGiOXml41nTx49fPz0+dY3savgkw+G1T/tWpHP8Xv7bUmnidfEGBOTc+pT/wXKG9k5M6QOagAAAABJRU5ErkJggg==' width='56' height='56' />"

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAABVElEQVR4nK2SzU/CQBDFO7vbisWW2qIpoDFI1PgR/f/PRuPBq4kHY5BECkKLlBbo144VNbSUcHJPO/vLm5m8t0TYcMg/QSCEAEARApHUxtnV5UXrwJAZrCiJqNZPr2/OW6ZWYgSyMC0IK+umaSgSCIirMxFERd+VhSgI+YKSpRA5iuqeSgPPj5LCtoiiYe4EY3cW59viAm7XamRsTyKORaVQaeqzvjNLcI0JtH4k2pYbY9EhFOQTPez0gz+WUzKzKTqvk2QdJMqxHrV7UeZleSuZh8x58XENBKY1quHbewxCIRWgsrEvOe0xoSmFHAS6ValWeM8KGP0OBDIQiFjWNMnvjxKWpgW5tkAlWSlx1/Y4oyD8uPkLAQiTKPdHQy9e5JNfCHk4dbqdjuVMw4QLWW+RR571/Hh/e/fw1J0mmFemdO7ag+HH4HO+dC+fCiJmy82f+gsv8o85LZRlVAAAAABJRU5ErkJggg==' width='56' height='56' />"

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAACKElEQVR4nI2SXW/SUBiAe057aDsLHaWC7aIbIS4RQvxCXeLiIjGbP8Arf4Be6U/apfHeZMYYY7xwMYIymDOyDMY2TCgwhkALbU977ADHYozZuX3ynPfNkxdS/3nwTBBAmkHI50M0BH9ByHBiJBq/efvW1cuKyNHgFIRICMdSK4+fPn/25NFSQhEYcAIBMyXPJReW7qcfrCw/XEzOSuwpiM7J6sUZmXMHNidH59QgD08gZHieJbq2k8/mSh0UlAIcPZkJAO7VK5uZ9fUv2y2K532TmYAiVlerFDdzucJOfUAxEJDJtgQbLa1aLu3uN3QX0pRjYQLGkHiw02o2Gu0+NSWwxDYMy52YjmUOBqbLBCKK5MPdjm4RAP4sRLwMvHgheiUelRn98MhwIPTouBDrxUuk7t5biIepbquLwbDwcFvaC5RILaaX03digtPVXcRzPnpkAlaKxq+nblxLzqsBhB3GH5z2c2hk0kIkOh+bkUUOOpZF8bKiRoLH7Y8hI4RVJSQGfJRlGDaSlEuzaoj3QgxNThCliHJ+GpntVtdhpXDIj4g7/BYA4BAkyiLq1/bK+3XdxlbfMPHIdLx4WqNZK2/lc/mt4napvPfzsIeHJrGPDr59ev96be3Nu4+fs9lMNl88aOpjE/equbcvV1dfvPqQyRc2vua+72q/Bu44vK03SoVsZuNHpVrTalq92daH5cdtXQfblmWatm1jGzuOS/5x1MR7Z7z43yol+TnIcFuAAAAAAElFTkSuQmCC' width='56' height='56' />"

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAACIElEQVR4nKXSzU8TQRgG8M7u7IfbslsWoZVt0VRslNKiJI0pKUYxEsFoTODi0buJN2P8BzTx4MWzJ49GIxghosbEKGIKEUqI1YqtabJNWyit3Y92P8aaslu86MG5/vLMzPvMYI6/LOz/EWA4QdE0TZEQA38gBplO38DJMxOT50ZCfV0MxNqIU24hdOry1es3b924dmUsIrAQWIgRzq6DoeHo8WP9R0+MjMYGD/EkZiGAlNNJmuXM2sdEahu6e3gnYScBQI1Kbn1x4dnM/OLXHQRx0D4TGWpFTCeXl1e/5CUDGLKkaMhGTa2W8mKhBj39QR9VKxSqjTaamqqoiDscGz873KMXxJK0J2maBqK94bEL41E/o0lyXXegdgkIQNeBYOhIN67WCb7TRQHQxt+zEqiaTa6si7gQ6GXJPaNgACnF1Pu5JzOv1iodfh/P4KCFAOA4jmQx+fbFwpuljSLgeI6GVrL5HFRz03z2eyYrbssGTlJ2fQDu63BzLtKha5oJXTSGzNZ9WsjwHqHXu59jeSEQ8BCqWtcMe1uK9QYGh8LhoWg8HunWt7YqsoF20TQA64+MTlyamr4Y7zNymz+KkpVEmvJTcnDB2PnJ0wNMfjWRzJbtbpEmlbKpz+n8Tjn36eXs/Id0UTFtNJRyJvH84f27t+/cezC79K2ktqxVvK6Ucxvv5h4/evp6ZbMk67tmFW/qDVWu1SSlOYRF//jUvwCWNPI5yQz3eAAAAABJRU5ErkJggg==' width='56' height='56' />"

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAB3ElEQVR4nK2SS2/TQBSFPWN7bCd2kk7t2omTorYskAAJlogFLGDBL4YNoLJDQiyKqEQSUGoam0dD/H7U9gyOm7gRC9hwl/fo3LnfnQOZvxT8XyKoC0IAtsWqwXI8z3FIbCtKW+LhRgSQ48XODsY9pbNrHhzcGKgdBDdOyEmKao5Mratg8/BouNeTeNA4eWnHGI76u7Ika6apyjyg67HVBpzUM8xB1YQSNlQZ5llBr99s4/5AazNZKWh6F2Zx0ogMFOS9gd7KXZ8ohipkQZiRxsmK2DR75cXPFOnDbun7SUkbFE7W9/vIX+SSftNEceWrr3AlIjzcVxHsjG7dv2uwOcPxLFyjAFbCutbFRw+ePntyG1c3aoloyymyhMF3Hj5+dE9nSwJY9nosKUN7PLaDAiCU/Zg7y6jGrEVahucnr5+/ePnm/WdrevJhartxXm5QynB++vb41fG78Wzy8XRiL+PLBoXS1HWss6/fvGhpW84yzgtCG05aXqZJWkDERIsLL61dzWdTSgjDCi2BRp4XF6swMM22DF3dXkAgi8IoLehVSLYyBCAgaeC53gqD/iHSIna/z88ta74IknxroUoieew6sy+TT9OZ8yvKCd12UlJkSRx4fphka5B/hPo3YxrdOQCsxRYAAAAASUVORK5CYII=' width='56' height='56' />"

...

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAB7UlEQVR4nJ2SSW/TQBiGPeOxx7HjrV6SdCEpBFIEIhSoWgkBv4pDr9w48yN6QUgghAQHJDggKg5sAglVhSYNURK1tkNrYjseD0lJarOIA99ppEfvO++3QOYfBY9fAMCfBQD4DQKIsKgXy9VatVyQBRZkIIAsJ6rmiXMra6sXT1oSD8ExHDkhPieb85XFSknLcRPrqS1ksajYpYKGAs/zI8L8YssJklm0Zeq0Wr2DkCQ0A1k+p9mWyvZ3G20vIJRmlQjLhm0IfqvR+RYlI5YqwTirbWtJp9E5pAghNhuIxXmjaPFusxtiRVUkjFI4yqoUikrY2UuM8qnFOVPG40YntkjQChbf34fl5dUrF87M6yKX/okk01bjPqxevb5Wr1VmZ0QuVXLSjIUDUlq+VDXyuqUrAoKpUlRlSKTyAnbbDsnl+aO4EwhxjmexKgef328fIp4m8XhERxBQSgEranLcabq8pTPfDwbDKWTocOATZdbiobZ0+TTndPf9KJnCJHC7DjdXW6pfu3EWNLd29vx4qqRk0Nv62GAW6ivntd7rl2+aXpTOlkbOp+ePnn5ot94+uffgxbYTZlZGY//r5sbt9Zvrt+7c3dzphyS7MoaE7pdXj+8/fPZu15ugzGlSSuIoGARDklD6x93+pf4f/gCy3rs5u68bUQAAAABJRU5ErkJggg==' width='56' height='56' />"

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAB6UlEQVR4nMWSzWsTURTF33uZZNJMR9ORxlTTIVNFDK6kunGpOxcKrroQxYLgoviPWbCCC3EpCKKEaqUNxJa0nSZxMpM0852Z9+Y9Z5pJ2rpw693+OJdzz7kI/GPQ/4MQosx4EILwHIQcLxTnr8iKsqQo8oI0m+dgCmEmX7x68+6DxyvPXr5ae722+uRe9SKPTmDMLpSvLd9/tPJ89UUyTx/eqRazcKxEOaFYqihLckkAQchJcvWymIOnhiLiD7vNLx/evnn3+TBAjOCIjSHDI0s/2tv69H59fePjDz2wzaFHUkixZxnd/e361/rPXR3MkEGn76VKRiPsW0Z7v6UadlSUJa/dMUOWGgKUhL5paAOfZaVaTRgcGSMwMcRYRALX9Ei2IN1YrmC1bUcATJSAkXA0IignVm7Xcmqz4xAKUkOxlOIQE4pmlVvSYOfXbzeM6PTOxBMOSebS9TLebRwejwhlbAoZpRFF4mIl323s6R6OKDtNiCXNiAvynNtq9Zzw5MgphABlxdLiPOipmhVM2KRsxBWkcol3NM304+QYO1M24vLinDRDhvqxg+lfbwIzfEHgI7tvmD6m7PxaFj8RtvV2WxvEdsA4n+mdxDUOdr5vbjXVvpt0yc4qsdNrbde/bTYOdCuJYLz2D1YkBoAf1olrAAAAAElFTkSuQmCC' width='56' height='56' />"

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAB/ElEQVR4nK2SPW/TQBzG7Ts7fkmdOAkkCgkVaUEF2ggQQwkSL2FBCPEBWJlZGZD4Fqgb6oAQEwsSCwixwIBUFIR4KaWhSkpTQhMTp45f4jvfYVo7TgVi4u/xp5+f+z93gPnHgP8F2Z35A7KAl9R8aebodCGjiBxgI+ijieyx6vWbt+/cunHl1GRKhCCELBCSxfKFq9cun69Uzp45MZVVYr4LRl5prlyK95v1Fk5kc2lFYJkQCkqmUEwO66+evqi1sBiPC9FvIS8oSRm13i3V6prL+oD6X2AyAHKQcfVOD4npFD80DAuFkDLEQxizUjI3NTujmu3NjuGSEBJkD3QT5ufmL1byxtrqRs9GhAaZnmv1tjSmOH/p3EHj88eGZiESlkAptvXuT6wens0Nvnxa79mY0DDTpx5yCZQFqjXWNduje7plOUnNpklreaXtAA6C3faD+jh5X+nQRLv2ZnUbSGJst/dgTyiohcnM4P3S8pYDJZGHMIIMlFK5NG6uNLoW5WK++Hv3kSkm4sDWbSjLMX4HUWYsEwLK7z9y8ngxwVOEkUdGJiXYsb1UuVo9fYCz+9umi0lkIqO7+V3DojT80fja7AxcL9qTYmPjw8snDxfv3X/0vNbwCxrPpK6+9vrx4t2FB8/efus7mNDxhqh/Z45lmpYTnOVv73bv/ALTAeQ5Ie+StwAAAABJRU5ErkJggg==' width='56' height='56' />"

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAB2ElEQVR4nL2ST2vUQBjGM5NNutlsNsvWbddqCypVCqWoLYhYLyIIvXrz5EFP4kER+nXEs2cVP0AFaSm0ympLzdY16Tbd7GSTbLL5MzNm29RJQTw61x/P+z7zPC/k/vHg/4AgfX+FAArFUlmRpbECBGch4Iu1S7cePlt99fj+XL3IgxyEgnJ+YeXJi5fPnz56cHNGFY/FMGNyY/7eyu263TLBxIV6RWTKdOb45cWlSf3d6zcf97mKMpYbmwonrl6rtD68fb+hJ3JVgswQgGK5MV1D62s7jlCfbsggIZQpC6WqSg2tL8/MX59VIydIKGU7BYGPw9KVxbvLN6YS88hPmJISHAVxZe7O8tLsJDRb1hCzEGgSINORpi7WeEjRj19uTJmSxI6xu9M2dc0YuPuaFRLmlpLI6zTXP619blpBZ8/w8YkwG4uHqP1ta3sXUbetoYjkW6EkGbrdQysS+f7Pjkc4eqYVgpMY80qVsw5QujENjwU/+mtBqp6TQ6s3iAmXdX4C6Yip4yoc2G6IT1kGQZqRVFaEyHX8iPy5FXh6IwVBBKGD+t4QHxuiuZ2pYb93qOsHRz0niHCulRTFnql93dz80vy+Z6Agxjk4Eoae3e1atu36YZyl8BsrauM5qwPBrQAAAABJRU5ErkJggg==' width='56' height='56' />"


[0.0987169, 0.112365, 0.0993001, 0.102183, 0.0973671, 0.0903516, 0.0986336, ...]
<MXNet::NDArray 10 @cpu(0)>

In [18]:
log_xcount = MXNet::NDArray.log(xcount)
log_xcount_neg = MXNet::NDArray.log(1 - xcount)
log_py = MXNet::NDArray.log(py)

mnist_iter = MXNet::IO::MNISTIter.new(
  image: 't10k-images-idx3-ubyte',
  label: 't10k-labels-idx1-ubyte',
  batch_size: 1
)
mnist_iter.each_with_index do |batch, batch_i|
  x = batch.data[0].reshape([784])
  y = batch.label[0].to_i
  
  log_px = log_py.dup
  0.upto(9) do |i|
    log_px[i] += MXNet::NDArray.dot(log_xcount[0..-1, i], x) +
                   MXNet::NDArray.dot(log_xcount_neg[0..-1, i], 1-x)
  end
  log_px -= MXNet::NDArray.max(log_px)
  px = MXNet::NDArray.exp(log_px).to_narray
  px /= px.sum

  imshow(x.reshape([28, 28]).to_narray)
  puts px.to_a.map.with_index {|v, i| "[#{i}] %0.3f" % v }.join(', ')
  break if batch_i == 10
end

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAAu0lEQVR4nGNiwAOYqCUZ+L4YpyRP/qOFOCUL7Be8wSlpfmUxTjs1vTa/wSkZ/HMpbteab7qKU5JL7xYDTkkzuQ+4Jd0ZduKWZPj1E4/k7Tt4JEXK9pw508yBTdI9WDz/z5+HNbWYkrwTcj8xRHlY5DFwY0ha9p30qWLgZ2Bw+zENQ/Jm9nKGG59MGCwnXLyFIfnuFwPDoy59UQe+bhyuXWh3rPzYWhyST0JUDvji8ifDXsaAdzgl0QCNJAHNjDc5mKcHJwAAAABJRU5ErkJggg==' width='56' height='56' />"

[0] 0.000, [1] 0.000, [2] 0.000, [3] 0.000, [4] 0.000, [5] 0.000, [6] 1.000, [7] 0.000, [8] 0.000, [9] 0.000


"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAArklEQVR4nGNiwAOYBoVk9f+tuCV1/+nilBTzxmNsHRfDO5ySUQwMy3FIMnfwMVyehUPSpJSR4cx7HJJBQLwWh4NkcxkYpuzGIXmGneHP5j/YJRXYGRiO72HAKsm6kJeBIYwBu6SbDQPDqg/YJTkXMDBcSf6FXdJGiOFN7zcG7JI1DAy1ixiwS7pYMPTMY8Ahycfya/UfXJIMDJ1nGHBKfu9nwCm5jucjbkksAK8kAMiDJjkdj6C1AAAAAElFTkSuQmCC' width='56' height='56' />"

[0] 0.000, [1] 1.000, [2] 0.000, [3] 0.000, [4] 0.000, [5] 0.000, [6] 0.000, [7] 0.000, [8] 0.000, [9] 0.000


"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAAlklEQVR4nL3S2w3DIAwF0OssFnsy7MlwJqNAlIgWk49KLV+ggx9YbHhY2x9QS+EVagLyV2l3iPkCmeE7K8cI1H5SzhGm61LJM7blYlY5B+hgV6Ve4R0VOMjaTqKnVNczQVzzantG+2h7EWkrbO/QGc906Q4c0HsA1wmKRpi0DdY9aMh6Ssg9gxG1RTg5IoQQkQznX/3bF5jSKzlYiEuEAAAAAElFTkSuQmCC' width='56' height='56' />"

...

[0] 1.000, [1] 0.000, [2] 0.000, [3] 0.000, [4] 0.000, [5] 0.000, [6] 0.000, [7] 0.000, [8] 0.000, [9] 0.000


"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAA9UlEQVR4nGNiwAOYBrmkZuGMGQdfGWORtOs7faonJdVaeGsQuiR3zIF8Yy5GEBAVQZdctOA/HKSKoEhyXw0EanlcFzoJpNXkIIrkIvX//18XmbS9sQXp/PcfRRKo763DRJODB4xAOr/XokiC1C86vdUabGWryXoUSZAbjcG6GBlD626gurYV4dTWdeghVBt27vu58yB9u+owg2+dqanpBqC+67EMmJIMDDeMm4A+rX2DVZKhCqixbT0DVslqUBjNYsAqGVQBcikDdskWYIycm4RdMggYuP8932CXXPeW8fsyVDkkY4sOt8Ux4JJc4tDGgFMSCwAA+SBvOWv0Bp0AAAAASUVORK5CYII=' width='56' height='56' />"

[0] 0.000, [1] 0.000, [2] 0.000, [3] 0.000, [4] 0.000, [5] 0.000, [6] 0.000, [7] 0.000, [8] 0.000, [9] 1.000


"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAABMUlEQVR4nGNiwAOYBliSlR27pJzfwoV5azcvXDhdHwhQJXceWxcd3e/lFB2ddhYI1kkiS7pIgMj3Sz5cW3IfyPDfKIkkOZ1hTYqUlHqCulW8idV2BgYjHSTJHJaIBa9evmN4+4Xh48WdjEx/f2H3CueUCf//HTiIVZJ9UiIDw68u7IFwFijHkL4Xm2T8Ck0Ghmsqi7CEkMqqeaFMjBcrHjBgSuZtD/r//2m7zzYGTMmSfkUgaVPznAFDUqqtBUjeswEbybns/PnZSJI6ZSwMDD0mJxjESuY+exCmy9yKbCwjI9MbpprP/593Johzvlqt9wBJ8vP3//9Eioo4//3//26OhVQkip3HXS6C6V+3ZzumX0N37QkPQ/fAfhtju8wrWLzy+tLezaUnrr1GChUaJU0A7DZsOeW5kOwAAAAASUVORK5CYII=' width='56' height='56' />"

[0] 0.000, [1] 0.000, [2] 1.000, [3] 0.000, [4] 0.000, [5] 0.000, [6] 0.000, [7] 0.000, [8] 0.000, [9] 0.000
